In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from time import *
import os 
import pickle
import pandas as pd

In [2]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False  # image is unwritable
    results = model.process(image)  # make prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [3]:
path = "D:\Documents\Research\ASL_2024\data\DataCreation\[TEST] Collected"
print(path)
frames = []
# run all directories in data
for element in os.listdir(path):
# run for all files in each subdirectory in data
    for file in os.listdir(os.path.join(path,element)):
        # don't change this
        mp_holistic = mp.solutions.holistic #holistic model
        mp_drawing = mp.solutions.drawing_utils #drawing utility
        cap = cv2.VideoCapture(os.path.join(path,element,file))
        
        with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
            while cap.isOpened():
                ref, frame = cap.read()
                if ref is False:
                    break
            # Make Detections
                image, results = mediapipe_detection(frame, holistic)
                rh = np.array([[res.x, res.y, res.z] for res in
                            results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.full([21, 3], 0)
                lh = np.array([[res.x, res.y, res.z] for res in
                            results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.full([21, 3], 0)
                face = np.array([[res.x, res.y, res.z] for res in
                                results.face_landmarks.landmark]) if results.face_landmarks else np.full([468,3], 0)
                pose = np.array([[res.x, res.y, res.z] for res in
                                results.pose_landmarks.landmark]) if results.pose_landmarks else np.full([33,4], 0)
                # Draw Landmarks
                
                # Put all the landmarks into a 1d numpy array with the xyz coordinates for face, pose, rh and lh
                landmarks_split = [rh, lh, face, pose]
                frames.append(landmarks_split)
            
                draw_landmarks(image, results)
                
                cv2.imshow("OpenCV Capture", image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
        cap.release()
        cv2.destroyAllWindows()
    
all_frames = np.array(frames, dtype=object)
np.save('frames.npy', all_frames, allow_pickle=True)

D:\Documents\Research\ASL_2024\data\DataCreation\[TEST] Collected


In [4]:
np.load('frames.npy', allow_pickle=True)

array([[array([[ 4.71678764e-01,  7.42132723e-01, -5.60565852e-08],
               [ 4.99711245e-01,  7.30325341e-01,  4.51732473e-03],
               [ 5.28303504e-01,  7.36239791e-01,  3.57501884e-03],
               [ 5.49445033e-01,  7.43658721e-01,  8.07254866e-04],
               [ 5.66131771e-01,  7.42886424e-01, -1.54239207e-03],
               [ 5.40231824e-01,  7.65201390e-01, -2.36607878e-03],
               [ 5.55991292e-01,  8.18310201e-01, -6.14228146e-03],
               [ 5.49932420e-01,  8.32392633e-01, -6.98611187e-03],
               [ 5.43397963e-01,  8.30876827e-01, -6.45415764e-03],
               [ 5.30535579e-01,  7.85471737e-01, -7.79589824e-03],
               [ 5.45499980e-01,  8.43197465e-01, -8.44441820e-03],
               [ 5.36827505e-01,  8.50170910e-01, -6.28406741e-03],
               [ 5.29694855e-01,  8.39933038e-01, -5.04773995e-03],
               [ 5.17420113e-01,  8.03747177e-01, -1.27603346e-02],
               [ 5.32264531e-01,  8.57723951e-01